# Spatial Query Cookbook

Sample queries for both **SedonaSpark** and **DuckDB** against the Colorado sample data.

**Prerequisites:** Run `colorado_sample_data.ipynb` first to populate the Iceberg tables.

| Engine | Best for | Access |
|--------|----------|--------|
| SedonaSpark | Heavy ETL, joins across large tables, raster | This notebook (Jupyter) |
| DuckDB | Fast interactive queries, AI query loops | CLI: `docker compose exec duckdb duckdb -init /config/init.sql` |

---
## Part 1: SedonaSpark Queries

These cells run natively in this Jupyter environment via Spark SQL + Sedona spatial functions.

In [ ]:
from sedona.spark import SedonaContext

sedona = SedonaContext.builder().getOrCreate()

### 1.1 Basic catalog exploration

In [ ]:
# List all namespaces and tables in the catalog
sedona.sql("SHOW NAMESPACES IN lakehouse").show()
sedona.sql("SHOW TABLES IN lakehouse.colorado").show()

In [ ]:
# Inspect table schema
sedona.sql("DESCRIBE lakehouse.colorado.points").show(truncate=False)

### 1.2 Distance queries

In [ ]:
# Find the 10 nearest points to Denver (39.74, -104.99)
sedona.sql("""
    SELECT id, name, category,
           ROUND(ST_Distance(
               ST_GeomFromWKB(geometry),
               ST_Point(-104.99, 39.74)
           ) * 111.32, 2) AS dist_km
    FROM lakehouse.colorado.points
    ORDER BY dist_km
    LIMIT 10
""").show(truncate=False)

In [ ]:
# Points within 25km of Colorado Springs (38.83, -104.82)
sedona.sql("""
    SELECT id, name, category,
           ROUND(ST_Distance(
               ST_GeomFromWKB(geometry),
               ST_Point(-104.82, 38.83)
           ) * 111.32, 2) AS dist_km
    FROM lakehouse.colorado.points
    WHERE ST_Distance(ST_GeomFromWKB(geometry), ST_Point(-104.82, 38.83)) < 0.225
    ORDER BY dist_km
""").show(truncate=False)

### 1.3 Bounding box / envelope queries

In [ ]:
# Points inside a bounding box around Boulder area
sedona.sql("""
    SELECT id, name, category,
           ST_AsText(ST_GeomFromWKB(geometry)) AS wkt
    FROM lakehouse.colorado.points
    WHERE ST_Within(
        ST_GeomFromWKB(geometry),
        ST_GeomFromWKT('POLYGON((-105.35 39.95, -105.15 39.95, -105.15 40.10, -105.35 40.10, -105.35 39.95))')
    )
""").show(truncate=False)

### 1.4 Aggregation by category

In [ ]:
# Count points by category
sedona.sql("""
    SELECT category, COUNT(*) AS count
    FROM lakehouse.colorado.points
    GROUP BY category
    ORDER BY count DESC
""").show()

In [ ]:
# Polygon area statistics by parcel type
sedona.sql("""
    SELECT parcel_type,
           COUNT(*) AS count,
           ROUND(AVG(area_sqm), 0) AS avg_area_sqm,
           ROUND(MIN(area_sqm), 0) AS min_area_sqm,
           ROUND(MAX(area_sqm), 0) AS max_area_sqm
    FROM lakehouse.colorado.polygons
    GROUP BY parcel_type
    ORDER BY avg_area_sqm DESC
""").show()

### 1.5 Spatial joins

In [ ]:
# Find points that fall inside polygons (point-in-polygon join)
sedona.sql("""
    SELECT p.id AS point_id, p.name AS point_name, p.category,
           pg.id AS polygon_id, pg.parcel_type
    FROM lakehouse.colorado.points p
    JOIN lakehouse.colorado.polygons pg
      ON ST_Within(
           ST_GeomFromWKB(p.geometry),
           ST_GeomFromWKB(pg.geometry)
         )
    LIMIT 20
""").show(truncate=False)

In [ ]:
# Lines that intersect polygons
sedona.sql("""
    SELECT l.id AS line_id, l.name AS line_name, l.length_km,
           pg.id AS polygon_id, pg.parcel_type
    FROM lakehouse.colorado.lines l
    JOIN lakehouse.colorado.polygons pg
      ON ST_Intersects(
           ST_GeomFromWKB(l.geometry),
           ST_GeomFromWKB(pg.geometry)
         )
    LIMIT 20
""").show(truncate=False)

### 1.6 Geometry operations

In [ ]:
# Buffer points by ~1km and check overlap with polygons
sedona.sql("""
    SELECT p.id, p.name, p.category,
           COUNT(pg.id) AS overlapping_parcels
    FROM lakehouse.colorado.points p
    JOIN lakehouse.colorado.polygons pg
      ON ST_Intersects(
           ST_Buffer(ST_GeomFromWKB(p.geometry), 0.01),
           ST_GeomFromWKB(pg.geometry)
         )
    GROUP BY p.id, p.name, p.category
    HAVING COUNT(pg.id) > 1
    ORDER BY overlapping_parcels DESC
    LIMIT 10
""").show(truncate=False)

In [ ]:
# Compute convex hull of all points per category
# Note: ST_Collect is not an aggregate function in Sedona/Spark 4.0,
# so we collect geometries as arrays first, then compute the hull.
sedona.sql("""
    SELECT category,
           COUNT(*) AS point_count,
           ROUND(
               ST_Area(
                   ST_ConvexHull(
                       ST_Union_Aggr(ST_GeomFromWKB(geometry))
                   )
               ) * 111.32 * 111.32, 2
           ) AS hull_area_sqkm
    FROM lakehouse.colorado.points
    GROUP BY category
    ORDER BY hull_area_sqkm DESC
""").show()

### 1.7 Iceberg table metadata

In [ ]:
# Iceberg snapshot history
sedona.sql("SELECT * FROM lakehouse.colorado.points.snapshots").show(truncate=False)

# Iceberg file manifest (how data is stored in S3)
sedona.sql("""
    SELECT file_path, file_format, record_count, file_size_in_bytes
    FROM lakehouse.colorado.points.files
""").show(truncate=False)

---
## Part 2: DuckDB Queries

Run these in the DuckDB CLI:
```bash
docker compose exec duckdb duckdb -init /config/init.sql
```

DuckDB is ideal for fast, interactive queries. The spatial extension provides PostGIS-compatible functions.

### 2.1 Catalog exploration
```sql
-- List all tables across namespaces
SHOW ALL TABLES;

-- Describe a table
DESCRIBE lakehouse.colorado.points;
```

### 2.2 Basic spatial queries
```sql
-- Sample points with geometry as WKT
SELECT id, name, category,
       ST_AsText(ST_GeomFromWKB(geometry)) AS wkt
FROM lakehouse.colorado.points
LIMIT 10;

-- Count features in each table
SELECT 'points' AS tbl, count(*) AS n FROM lakehouse.colorado.points
UNION ALL
SELECT 'lines', count(*) FROM lakehouse.colorado.lines
UNION ALL
SELECT 'polygons', count(*) FROM lakehouse.colorado.polygons;
```

### 2.3 Distance and proximity
```sql
-- 10 nearest points to Fort Collins (40.59, -105.08)
SELECT id, name, category,
       ROUND(ST_Distance(
           ST_GeomFromWKB(geometry),
           ST_Point(-105.08, 40.59)
       ) * 111.32, 2) AS dist_km
FROM lakehouse.colorado.points
ORDER BY dist_km
LIMIT 10;

-- Lines within 10km of Grand Junction (39.07, -108.55)
SELECT id, name, length_km,
       ROUND(ST_Distance(
           ST_GeomFromWKB(geometry),
           ST_Point(-108.55, 39.07)
       ) * 111.32, 2) AS dist_km
FROM lakehouse.colorado.lines
WHERE ST_Distance(ST_GeomFromWKB(geometry), ST_Point(-108.55, 39.07)) < 0.09
ORDER BY dist_km;
```

### 2.4 Bounding box filter
```sql
-- Polygons in the Denver metro area
SELECT id, name, parcel_type, area_sqm,
       ST_AsText(ST_GeomFromWKB(geometry)) AS wkt
FROM lakehouse.colorado.polygons
WHERE ST_Within(
    ST_GeomFromWKB(geometry),
    ST_GeomFromText(
        'POLYGON((-105.2 39.6, -104.7 39.6, -104.7 39.9, -105.2 39.9, -105.2 39.6))'
    )
);
```

### 2.5 Aggregation
```sql
-- Average line length by road type (extracted from name)
SELECT split_part(name, '_', 1) AS road_type,
       COUNT(*) AS count,
       ROUND(AVG(length_km), 2) AS avg_length_km,
       ROUND(SUM(length_km), 2) AS total_length_km
FROM lakehouse.colorado.lines
GROUP BY road_type
ORDER BY total_length_km DESC;

-- Parcel count and total area by type
SELECT parcel_type,
       COUNT(*) AS count,
       ROUND(SUM(area_sqm) / 1e6, 2) AS total_area_sqkm
FROM lakehouse.colorado.polygons
GROUP BY parcel_type
ORDER BY total_area_sqkm DESC;
```

### 2.6 Spatial joins
```sql
-- Points inside polygons
SELECT p.id AS point_id, p.category,
       pg.id AS polygon_id, pg.parcel_type
FROM lakehouse.colorado.points p
JOIN lakehouse.colorado.polygons pg
  ON ST_Within(
       ST_GeomFromWKB(p.geometry),
       ST_GeomFromWKB(pg.geometry)
     )
LIMIT 20;

-- Count points per polygon
SELECT pg.id, pg.parcel_type, pg.area_sqm,
       COUNT(p.id) AS points_inside
FROM lakehouse.colorado.polygons pg
LEFT JOIN lakehouse.colorado.points p
  ON ST_Within(
       ST_GeomFromWKB(p.geometry),
       ST_GeomFromWKB(pg.geometry)
     )
GROUP BY pg.id, pg.parcel_type, pg.area_sqm
HAVING COUNT(p.id) > 0
ORDER BY points_inside DESC
LIMIT 15;
```

### 2.7 Geometry construction and export
```sql
-- Centroid of each polygon
SELECT id, parcel_type,
       ST_AsText(ST_Centroid(ST_GeomFromWKB(geometry))) AS centroid_wkt
FROM lakehouse.colorado.polygons
LIMIT 10;

-- Bounding box (envelope) of all points
SELECT ST_AsText(
    ST_Envelope(
        ST_Collect(ARRAY_AGG(ST_GeomFromWKB(geometry)))
    )
) AS extent
FROM lakehouse.colorado.points;

-- Export query results as GeoJSON features
SELECT json_object(
    'type', 'Feature',
    'properties', json_object('id', id, 'name', name, 'category', category),
    'geometry', ST_AsGeoJSON(ST_GeomFromWKB(geometry))::JSON
) AS geojson
FROM lakehouse.colorado.points
LIMIT 5;
```

### 2.8 Cross-table analysis
```sql
-- For each line, find the nearest point
SELECT l.id AS line_id, l.name AS line_name,
       p.id AS nearest_point, p.category,
       ROUND(ST_Distance(
           ST_Centroid(ST_GeomFromWKB(l.geometry)),
           ST_GeomFromWKB(p.geometry)
       ) * 111.32, 2) AS dist_km
FROM lakehouse.colorado.lines l
CROSS JOIN LATERAL (
    SELECT id, category, geometry
    FROM lakehouse.colorado.points
    ORDER BY ST_Distance(
        ST_Centroid(ST_GeomFromWKB(l.geometry)),
        ST_GeomFromWKB(geometry)
    )
    LIMIT 1
) p
LIMIT 15;
```